<a href="https://colab.research.google.com/github/ArkS0001/Memory-Agent/blob/main/Memory_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
pip install langchain-groq langmem langchain_huggingface sentence-transformers

In [9]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Flipkart` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Flipkart`


In [10]:
!export GROQ_API_KEY="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"


In [11]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={
        "dims": 384,  # Updated dimension for the Hugging Face model
        "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2",
    }
)

# Initialize the Groq API-based LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory capabilities.
# The agent is built with the Groq LLM and memory tools to manage and search memories.
agent = create_react_agent(
    llm,  # Using the Groq API instead of an OpenAI-based LLM
    tools=[
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

# --- Using the Agent ---

# 1. The agent processes a message and stores a memory (e.g., your preference for dark mode).
agent.invoke({
    "messages": [
        {"role": "user", "content": "Remember that I prefer dark mode."}
    ]
})

# 2. Later, when asked, the agent searches its memory and retrieves the stored preference.
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "What are my lighting preferences?"}
    ]
})

# The response should reflect the stored memory.
print(response["messages"][-1].content)
# Expected output: "You've told me that you prefer dark mode."


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "search\\_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "dark mode"\n\t\t}\n\t}\n}\n</tool-use>'}}

In [17]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={
        "dims": 384,  # Updated dimension for the Hugging Face model
        "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2",
    }
)

# Initialize the Groq LLM with tool calling disabled.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150,
    tool_choice="none"  # Disable automatic function calls
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

# Optionally, you can adjust the system prompt (if supported) to instruct the agent not to call functions.
# For example:
# agent.system_message = "You are a helpful assistant. Do not call external functions; just respond directly."

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "my name is aakarshit."}
        ]
    })
    print("Store response:", store_response["messages"][-1].content)
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What is my name?"}
        ]
    })
    print("Search response:", search_response["messages"][-1].content)
except Exception as e:
    print("Error during memory retrieval:", e)


/usr/local/lib/python3.11/dist-packages/langchain_groq/chat_models.py:364: UserWarning: WARNING! tool_choice is not default parameter.
                    tool_choice was transferred to model_kwargs.
                    Please confirm that tool_choice is what you intended.
  warnings.warn(


Store response: Hello Aakarshit! It seems like you've shared some tools related to managing and searching memories. These tools allow you to create, update, delete, and search for persistent memories that can help in maintaining context and user preferences across conversations.

You can create a new memory by calling the `manage_memory` function with the action set to "create" and providing the content you want to store. Similarly, you can update or delete a memory by specifying the memory ID and the desired action.

To search for relevant memories, you can use the `search_memory` function with a query string. You can also filter, limit, and offset the search results.

These tools can be helpful in developing
Search response: I don't have a name or personal identity. I'm here to assist you with information and tools based on the data provided to me.


In [14]:
store = InMemoryStore(
    index={
        "dims": 384,  # Dimension for "all-MiniLM-L6-v2"
        "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2",
    }
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Use your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

# --- Step 1: Store a Memory ---
# Provide a system instruction to ensure the agent uses the manage_memory tool.
store_response = agent.invoke({
    "messages": [
        {
            "role": "system",
            "content": (
                "When a user instructs you to 'remember' something, "
                "store that information using the manage_memory function and do not search for existing memories."
            )
        },
        {"role": "user", "content": "Remember that I prefer dark mode."}
    ]
})
print("Store response:", store_response["messages"][-1].content)

# --- Step 2: Retrieve the Stored Memory ---
# Provide a system instruction to perform a direct memory lookup.
search_response = agent.invoke({
    "messages": [
        {
            "role": "system",
            "content": (
                "When answering, search your memory for stored user preferences and return only the relevant detail."
            )
        },
        {"role": "user", "content": "What are my lighting preferences?"}
    ]
})
print("Search response:", search_response["messages"][-1].content)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "manage\\_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"action": "create",\n\t\t\t"content": "I prefer dark mode",\n\t\t\t"id": null\n\t\t}\n\t}\n}\n</tool-use>'}}

In [16]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

# Set up an in-memory store using a Hugging Face embedding model.
store = InMemoryStore(
    index={
        "dims": 384,  # Dimension for "all-MiniLM-L6-v2"
        "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2",
    }
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

# --- Step 1: Store a Memory ---
# Store the dark mode preference with an explicit id.
store_response = agent.invoke({
    "messages": [
        {
            "role": "system",
            "content": (
                "When a user instructs you to 'remember' something, call the manage_memory function with action 'create'. "
                "Include a unique id. For example, if the content is 'I prefer dark mode', set the id to 'dark_mode'."
            )
        },
        {"role": "user", "content": "Remember that I prefer dark mode."}
    ]
})
print("Store response:", store_response["messages"][-1].content)

# --- Step 2: Retrieve and Display Stored Memory ---
# Instruct the agent to search for all stored memories and display them succinctly.
display_response = agent.invoke({
    "messages": [
        {
            "role": "system",
            "content": (
                "When answering, retrieve all stored memories and display only the relevant details without extra explanation."
            )
        },
        {"role": "user", "content": "Display stored memory."}
    ]
})
print("Display stored memory:", display_response["messages"][-1].content)


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "manage\\_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"action": "create",\n\t\t\t"content": "I prefer dark mode",\n\t\t\t"id": "dark\\_mode"\n\t\t}\n\t}\n}\n</tool-use>'}}

In [18]:
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Set up an in‑memory store using a Hugging Face embedding model.
# Here we use "huggingface:sentence-transformers/all-MiniLM-L6-v2", which produces 384-dimensional embeddings.
store = InMemoryStore(
    index={
        "dims": 384,
        "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2",
    }
)

# Create memory management tools.
# We pass the store explicitly so that the tools know where to persist and retrieve memories.
manage_memory_tool = create_manage_memory_tool(namespace=("memories",), store=store)
search_memory_tool = create_search_memory_tool(namespace=("memories",), store=store)

# --- Step 1: Store a Memory ---
# Use the manage_memory tool to create a memory entry.
# We explicitly supply an ID ("dark_mode") along with the content.
memory_input = {
    "action": "create",
    "content": "I prefer dark mode",
    "id": "dark_mode"
}
store_response = manage_memory_tool.invoke(memory_input)
print("Store response:", store_response)

# --- Step 2: Display Stored Memory ---
# Use the search_memory tool to retrieve the stored memory.
search_input = {
    "query": "dark mode"
}
search_response = search_memory_tool.invoke(search_input)
print("Search response:", search_response)


ValidationError: 1 validation error for manage_memory
id
  Input should be a valid UUID, invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `r` at 3 [type=uuid_parsing, input_value='dark_mode', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/uuid_parsing

In [22]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={"dims": 384, "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2"}
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[create_manage_memory_tool( namespace=("memories",) ), create_search_memory_tool( namespace=("memories",) )],
    store=store,
)

# Set the system prompt to instruct the agent to use memory tools.
agent.system_message = "You are a helpful assistant. When a user makes a statement, use the manage_memory tool to store it as a memory. When asked a question, use the search_memory tool to find relevant memories and use them to answer the question."

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "I love dark , and my favourite car is BMW , my favourite colour is red"}
        ]
    })
    print("Store response:", store_response["messages"][-1].content)
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What is my car colour"}
        ]
    })
    print("Search response:", search_response["messages"][-1].content)
except Exception as e:
    print("Error during memory retrieval:", e)

Error during memory storage: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "manage_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"action": "create",\n\t\t\t"content": {\n\t\t\t\t"user_preference_1": "dark",\n\t\t\t\t"user_preference_2": "BMW",\n\t\t\t\t"user_preference_3": "red"\n\t\t\t}\n\t\t}\n\t}\n}\n</tool-use>'}}
Search response: Your car's color is not something I have stored in memory, so I cannot provide that information directly. However, I can manage memory for you. If you'd like me to remember the color of your car for future conversations, you can use the 'manage\_memory' tool. Here's an example of how to use it:

<tool-use>
{
	"tool\_call": {
		"id": "pending",
		"type": "function",
		"functi

In [24]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={"dims": 384, "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2"}
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[create_manage_memory_tool( namespace=("memories",) ), create_search_memory_tool( namespace=("memories",) )],
    store=store,
)

# Set the system prompt to instruct the agent to use memory tools.
agent.system_message = "You are a helpful assistant. When a user makes a statement, use the manage_memory tool to store it as a memory. When asked a question, use the search_memory tool to find relevant memories and use them to answer the question."

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "Remember that I prefer dark mode"}
        ]
    })
    print("Store response:", store_response["messages"][-1].content)
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What are my lighting preferences?"}
        ]
    })
    print("Search response:", search_response["messages"][-1].content)
except Exception as e:
    print("Error during memory retrieval:", e)

Error during memory storage: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "search\\_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "dark mode"\n\t\t}\n\t}\n}\n</tool-use>'}}
Error during memory retrieval: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "search\\_memory"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "lighting preferences"\n\t\t}\n\t}\n}\n</tool-use>'}}


In [25]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={"dims": 384, "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2"}
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[create_manage_memory_tool( namespace=("memories",) ), create_search_memory_tool( namespace=("memories",) )],
    store=store,
)

# Set the system prompt to instruct the agent to use memory tools.
agent.system_message = "You are a helpful assistant. When a user makes a statement, use the manage_memory tool to store it as a memory. When asked a question, use the search_memory tool to find relevant memories and use them to answer the question."

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "Remember that I prefer dark mode"}
        ]
    })
    print("Store response:", store_response["messages"][-1].content)
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What are my lighting preferences?"}
        ]
    })
    print("Search response:", search_response["messages"][-1].content)
except Exception as e:
    print("Error during memory retrieval:", e)

Error during memory storage: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n"tool\\_call": {\n"id": "pending",\n"type": "function",\n"function": {\n"name": "search\\_memory"\n},\n"parameters": {\n"query": "dark mode"\n}\n}\n}\n</tool-use>'}}
Error during memory retrieval: Error code: 400 - {'error': {'message': "'messages.2' : for 'role:tool' the following must be satisfied[('messages.2.content' : value must be a string)]", 'type': 'invalid_request_error'}}


In [30]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={"dims": 384, "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2"}
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[
        create_manage_memory_tool( namespace=("memories",) ),
        create_search_memory_tool( namespace=("memories",) )
    ],
    store=store,
)

# Set the system prompt to instruct the agent to use memory tools.
agent.system_message = "You are a helpful assistant. When a user makes a statement, use the manage_memory tool to store it as a memory. When asked a question, use the search_memory tool to find relevant memories and use them to answer the question."

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "my name is aakarshit."}
        ]
    })
    print("Store response:", store_response["messages"][-1].content)
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What is my name?"}
        ]
    })
    print("Search response:", search_response["messages"][-1].content)
except Exception as e:
    print("Error during memory retrieval:", e)

Store response: Hello Aakarshit! It's nice to meet you. Let's have a conversation. Do you have any questions or topics you'd like to discuss? I'm here to help and provide information on a wide range of subjects.
Search response: Your name is not mentioned in the instructions provided.


In [31]:
import os
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Ensure your Groq API key is set in the environment.
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "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"

# Set up an in-memory storage for memories using a Hugging Face embedding model.
# "huggingface:sentence-transformers/all-MiniLM-L6-v2" produces 384-dimensional embeddings.
store = InMemoryStore(
    index={"dims": 1536, "embed": "huggingface:sentence-transformers/all-MiniLM-L6-v2"}
)

# Initialize the Groq LLM.
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Replace with your preferred Groq model name
    temperature=0.7,
    max_tokens=150
)

# Create an agent that integrates memory tools.
agent = create_react_agent(
    llm,
    tools=[create_manage_memory_tool( namespace=("memories",) ), create_search_memory_tool( namespace=("memories",) )],
    store=store,
)

# Set the system prompt to instruct the agent to use memory tools.
agent.system_message = """
You are a memory-enhanced assistant. You have two tools: manage_memory and search_memory.

- manage_memory: You must use this tool for any user input that is a statement (ends with a period or exclamation mark). The input to this tool is the user's statement, which you will store as a memory. Do not respond directly to the user; just call the tool.

- search_memory: You must use this tool for any user input that is a question (ends with a question mark). The input to this tool is the user's question, which you will use to search for relevant memories. After getting the search results, use the information from those memories to answer the user's question.

Your task is to process user inputs and use these tools accordingly.

For example:

- User: my name is aakarshit.

- You: Call manage_memory with {'content': 'my name is aakarshit.'}

- User: What is my name?

- You: Call search_memory with {'query': 'What is my name?'}, get the relevant memories, and then answer the question using the information from those memories.

Remember to always use the tools based on the type of user input and to formulate your answers based on the memories retrieved.

Do not provide any direct responses without using the tools as specified.
"""

# --- Using the Agent with error handling ---

try:
    # 1. Store a memory: the agent processes a message and (via its memory tool) stores the detail.
    store_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "my name is aakarshit."}
        ]
    })
    print("Store response messages:", store_response["messages"])
except Exception as e:
    print("Error during memory storage:", e)

try:
    # 2. Retrieve the stored memory: when asked, the agent should recall that detail.
    search_response = agent.invoke({
        "messages": [
            {"role": "user", "content": "What is my name?"}
        ]
    })
    print("Search response messages:", search_response["messages"])
except Exception as e:
    print("Error during memory retrieval:", e)

Store response messages: [HumanMessage(content='my name is aakarshit.', additional_kwargs={}, response_metadata={}, id='2beed77f-384e-4af4-85ad-8ea6c079d0c5'), AIMessage(content="Hello Aakarshit! It's nice to meet you. Is there anything specific you would like to talk about or ask me? I'm here to help! If needed, I can use various tools to provide you with accurate and updated information.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1917, 'total_tokens': 1970, 'completion_time': 0.080681621, 'prompt_time': 0.196031214, 'queue_time': 0.021217771999999996, 'total_time': 0.276712835}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b77b4ac-d776-4d09-8fa7-3534e21f2e77-0', usage_metadata={'input_tokens': 1917, 'output_tokens': 53, 'total_tokens': 1970})]
Search response messages: [HumanMessage(content='What is my name?', additional_kwargs={}, response_meta